### Merging 

In [8]:
import pandas as pd
import glob
import re

# -------------------------
# 1. Define columns to drop early
# -------------------------
drop_cols = [
    'OrgaanCode','OrgaanOmschrijving','OrganismeNaam','OrganismeNaam_Nederlands',
    'BemonsteringsapparaatCode','BemonsteringsapparaatOmschrijving','TyperingCode',
    'TyperingOmschrijving','GrootheidCode','GrootheidOmschrijving','ParameterCode',
    'ParameterCASnummer','BiotaxonNaam','BiotaxonNaam_Nederlands','EenheidCode',
    'EenheidOmschrijving','HoedanigheidCode','HoedanigheidOmschrijving',
    'LevensstadiumCode','LevensstadiumOmschrijving','LengteklasseCode',
    'LengteklasseOmschrijving','GeslachtCode','GeslachtOmschrijving',
    'VerschijningsvormCode','VerschijningsvormOmschrijving','LevensvormCode',
    'LevensvormOmschrijving','GedragCode','GedragOmschrijving','AnalyseCompartimentCode',
    'AnalyseCompartimentOmschrijving','WaardebewerkingsmethodeCode',
    'WaardebewerkingsmethodeOmschrijving','WaardebepalingsmethodeCode',
    'WaardebepalingsmethodeOmschrijving','LocatieTypeWaardeBepalingID',
    'LocatieTypeWaardeBepalingOmschrijving','Alfanumeriekewaarde',
    'KwaliteitsoordeelCode','KwaliteitsoordeelOmschrijving','PublicatieDatumTijd'
]

# -------------------------
# 2. Load and clean meetwaarden into a single DataFrame
# -------------------------
waarden_list = []
for file in glob.glob('Meetwaarden/Waterschap Brabantse Delta/WKP_Meetwaarden_Waterschap Brabantse Delta_*.csv'):
    # Extract year from filename
    jaar = int(re.search(r'_(\d{4})_', file).group(1))
    # Read and drop unwanted columns immediately
    df = pd.read_csv(file, sep=';', encoding='utf-8')
    df.drop(columns=drop_cols, errors='ignore', inplace=True)
    df['year'] = jaar
    waarden_list.append(df)
# Concatenate all years into one DataFrame
df_waarden = pd.concat(waarden_list, ignore_index=True)
print(f"Loaded and cleaned meetwaarden, shape={df_waarden.shape}")

# -------------------------
# 3. Merge meetwaarden with cluster info (keep all meetwaarden rows)
# -------------------------
df_clusters = pd.read_csv('meetpunten_met_clusters_relevant.csv')
# Prepare mapping of MeetobjectCode to cluster_id and relevant flag
df_cluster_map = (
    df_clusters[['MeetobjectCode','cluster_id','relevant']]
    .drop_duplicates(subset=['MeetobjectCode'])
)
# Left join preserves all meetwaarden rows
df_merged = df_waarden.merge(
    df_cluster_map,
    on='MeetobjectCode',
    how='left'
)
print(f"After merging clusters, rows={len(df_merged)} (should equal meetwaarden count)")

# -------------------------
# 4. Merge with forest mapping (one record per cluster)
# -------------------------
df_forest = pd.read_csv('meetpunten_met_clusters_relevant_met_bos.csv')
# Keep nearest forest per cluster_id if multiple
if 'distance_m' in df_forest.columns:
    df_forest_unique = (
        df_forest.sort_values('distance_m')
                 .drop_duplicates(subset=['cluster_id'], keep='first')
    )
else:
    df_forest_unique = df_forest.drop_duplicates(subset=['cluster_id'])
# Merge forest info into df_merged
df_final = df_merged.merge(
    df_forest_unique[['cluster_id','Naam','Grootte','Eerste aanplant','distance_m']],
    on='cluster_id',
    how='left'
)
print(f"After merging forests, rows={len(df_final)} (should still equal meetwaarden count)")

# -------------------------
# 5. df_final now contains:
#      - all original meetwaarde columns minus dropped ones
#      - year, cluster_id, relevant flag
#      - forest info (Naam, Grootte, Eerste aanplant, distance_m)
# Further analysis or save to CSV
# df_final.to_csv('df_final_slender.csv', index=False)

df_final.head(10)


/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_29980/384944011.py:33: DtypeWarning: Columns (4,7,8,9,15,16,17,25,26,34,35,40,41,42,43,48,49,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_29980/384944011.py:33: DtypeWarning: Columns (4,7,8,9,16,17,22,34,35,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_29980/384944011.py:33: DtypeWarning: Columns (4,7,8,20,24,25,26,34,35,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_29980/384944011.py:33: DtypeWarning: Columns (4,7,8,20,24,25,26,34,35,42,43) have mixed types. Specify dtype option on import or s

Loaded and cleaned meetwaarden, shape=(1115673, 25)
After merging clusters, rows=1115673 (should equal meetwaarden count)
After merging forests, rows=1115673 (should still equal meetwaarden count)


,Meetjaar,WaterbeheerderCode,WaterbeheerderNaam,MeetobjectNamespace,MeetobjectLokaalID,MeetobjectCode,Namespace,MonsterIdentificatie,MonsterLokaalID,MonsterCompartimentCode,...,ParameterOmschrijving,Limietsymbool,Numeriekewaarde,year,cluster_id,relevant,Naam,Grootte,Eerste aanplant,distance_m
0,2023,25,Waterschap Brabantse Delta,NL25,203602,NL25_203602,NL25,NL25_203602_594406OW_44762,203602_594406OW_44762,OW,...,Chroococcus microscopicus,NaN,377210.0,2023,87,True,NaN,NaN,NaN,NaN
1,2023,25,Waterschap Brabantse Delta,NL25,203602,NL25_203602,NL25,NL25_203602_594406OW_44762,203602_594406OW_44762,OW,...,Volvocales,NaN,9823.0,2023,87,True,NaN,NaN,NaN,NaN
2,2023,25,Waterschap Brabantse Delta,NL25,203602,NL25_203602,NL25,NL25_203602_594406OW_44762,203602_594406OW_44762,OW,...,Trachydiscus lenticularis,NaN,1965.0,2023,87,True,NaN,NaN,NaN,NaN
3,2023,25,Waterschap Brabantse Delta,NL25,203602,NL25_203602,NL25,NL25_203602_594406OW_44762,203602_594406OW_44762,OW,...,Trachelomonas,NaN,109.0,2023,87,True,NaN,NaN,NaN,NaN
4,2023,25,Waterschap Brabantse Delta,NL25,203602,NL25_203602,NL25,NL25_203602_594406OW_44762,203602_594406OW_44762,OW,...,Tetrastrum heteracanthum,NaN,7859.0,2023,87,True,NaN,NaN,NaN,NaN
5,2023,25,Waterschap Brabantse Delta,NL25,203602,NL25_203602,NL25,NL25_203602_594406OW_44762,203602_594406OW_44762,OW,...,Tetrastrum,NaN,17682.0,2023,87,True,NaN,NaN,NaN,NaN
6,2023,25,Waterschap Brabantse Delta,NL25,203602,NL25_203602,NL25,NL25_203602_594406OW_44762,203602_594406OW_44762,OW,...,Tetraedron minimum,NaN,1965.0,2023,87,True,NaN,NaN,NaN,NaN
7,2023,25,Waterschap Brabantse Delta,NL25,203602,NL25_203602,NL25,NL25_203602_594406OW_44762,203602_594406OW_44762,OW,...,Stichococcus,NaN,1965.0,2023,87,True,NaN,NaN,NaN,NaN
8,2023,25,Waterschap Brabantse Delta,NL25,203602,NL25_203602,NL25,NL25_203602_594406OW_44762,203602_594406OW_44762,OW,...,Spermatozopsis similis,NaN,3929.0,2023,87,True,NaN,NaN,NaN,NaN
9,2023,25,Waterschap Brabantse Delta,NL25,203602,NL25_203602,NL25,NL25_203602_594406OW_44762,203602_594406OW_44762,OW,...,Spermatozopsis exsultans,NaN,13752.0,2023,87,True,NaN,NaN,NaN,NaN
